In [1]:
import osmium
import sys
from neo4j import GraphDatabase, basic_auth, unit_of_work
from copy import deepcopy

In [2]:
url = 'http://download.geofabrik.de/europe/monaco-latest.osm.pbf'

In [3]:
class GraphNode:

    def __init__(self, node_id, lat, long):
        self.node_id = node_id
        self.lat = lat
        self.long = long

    def __repr__(self):
        return f'{{node_id: {self.node_id}, lat: {self.lat}, long: {self.long}}}'

In [4]:
class GraphEdge:

    def __init__(self, start_node_id, end_node_id, distance, rating):
        self.start_node_id = start_node_id
        self.end_node_id = end_node_id
        self.distance = distance
        self.rating = rating

In [13]:
class Neo4jHandler():
    
    def __init__(self):
        self.driver = GraphDatabase.driver(
            "bolt://54.167.190.221:7687",
            auth=basic_auth("neo4j", "release-shows-breads"))
        self.nodes = []
        self.edges = []
        
    def add_node(self, node):
        self.nodes.append(node)

    def add_edge(self, edge):
        self.edges.append(edge)

    def create_objects(self):
        with self.driver.session() as session:
            batch = []
            for i, node in enumerate(self.nodes):
                if (i % 5000 == 0 and len(batch) > 0) or i == len(self.nodes):
                    with session.begin_transaction() as tx:
                        tx.run(f'''FOREACH (node IN [{','.join(map(lambda x: str(x), batch))}]|
                               CREATE (:Node{{node_id:node.node_id,lat:node.lat,long:node.long}}))''')
                        tx.commit()
                        print(f'Close transaction {i//5000}')
                        batch = []

                print(i)
                batch.append(node)
            for i, edge in enumerate(self.edges):
#                 if (i % 5000 == 0 and len(batch) > 0) or i == len(self.edges):
                print(edge.start_node_id, '->', edge.end_node_id)
                with session.begin_transaction() as tx:
                    tx.run(f'''MATCH (a: Node), (b: Node)
                    WHERE a.node_id = {edge.start_node_id} AND b.node_id = {edge.end_node_id}
                    CREATE (a)-[r:Route]->(b)
                    ''')
                    tx.commit()
#                     print(f'Close transaction {i//5000}')
#                         batch = []
#                 batch.append(edge)
    
    def close(self):
        self.driver.close()

In [11]:
class FileHandler(osmium.SimpleHandler):

    def __init__(self, neoHandler):
        super(FileHandler, self).__init__()
        self.neoHandler = neoHandler

    def node(self, n):
        id = deepcopy(n.id)
        lat, lon = deepcopy(str(n.location)).split('/')
        node = GraphNode(n.id, lat, lon)
        self.neoHandler.add_node(node)

    def way(self, w):
        distance = osmium.geom.haversine_distance(w.nodes)
        if w.nodes[0].ref == w.nodes[-1].ref:
            return
        edge = GraphEdge(w.nodes[0].ref, w.nodes[-1].ref, distance, 4)
        self.neoHandler.add_edge(edge)
        

#     def relation(self, r):
#         print(r)


In [14]:
%%time
neoHandler = Neo4jHandler()
h = FileHandler(neoHandler)

h.apply_file(url, locations=True)

neoHandler.create_objects()
neoHandler.close()

Exception ignored in: <function Session.__del__ at 0x7ff4fe2834c0>
Traceback (most recent call last):
  File "/home/victor/miniconda3/envs/hacktues/lib/python3.9/site-packages/neo4j/work/simple.py", line 95, in __del__
    self.close()
  File "/home/victor/miniconda3/envs/hacktues/lib/python3.9/site-packages/neo4j/work/simple.py", line 149, in close
    self._transaction.rollback()  # roll back the transaction if it is not closed
  File "/home/victor/miniconda3/envs/hacktues/lib/python3.9/site-packages/neo4j/work/transaction.py", line 156, in rollback
    self._connection.send_all()
  File "/home/victor/miniconda3/envs/hacktues/lib/python3.9/site-packages/neo4j/io/_bolt4.py", line 296, in send_all
    raise ServiceUnavailable("Failed to write to closed connection {!r} ({!r})".format(
neo4j.exceptions.ServiceUnavailable: Failed to write to closed connection IPv4Address(('54.167.190.221', 7687)) (IPv4Address(('54.167.190.221', 7687)))


21912089 -> 21912099
21918500 -> 1868736741
25177418 -> 25177397
25178088 -> 25178088
25192130 -> 1074585009
1074584578 -> 1074584578
25201041 -> 25201047
25182821 -> 7932125562
25182815 -> 25182821
4938436902 -> 1079045416
25193709 -> 25194196
6510744220 -> 6510744220
25181935 -> 2737814240
96050956 -> 25182815
25182446 -> 2153445075
25181793 -> 25181806
25191634 -> 25191338
25216581 -> 25191243
25191904 -> 1573123086
25191508 -> 25191502
1737389117 -> 477618046
25192216 -> 1074585009
25194260 -> 25194196
4058432473 -> 4058432482
1690130866 -> 1738360319
6444966511 -> 25193390
1918966551 -> 4940692951
1074584672 -> 4453073043
25177397 -> 25177819
25204258 -> 25177418
937988290 -> 1866510571
1868723951 -> 21918450
1720684257 -> 21912962
25197949 -> 25197962
25198656 -> 25238669
25238772 -> 25197962
25198559 -> 25238728
25238712 -> 25197985
2422123078 -> 1720683756
25239325 -> 25239343
25239346 -> 25239325
21912099 -> 25239389
21913117 -> 25240075
25192169 -> 25192010
25238111 -> 252042

4800310686 -> 4800310686
1095426303 -> 1095426303
1095425613 -> 1095425613
1095426058 -> 1095426058
1095425526 -> 1095425526
1095426787 -> 1095426787
624435735 -> 624435735
1095426850 -> 1095426850
1095426449 -> 1095426449
1095426812 -> 1095426812
1095426352 -> 1095426352
1095426668 -> 1095426668
1095426716 -> 1095426716
1095426604 -> 1095426604
1095425677 -> 1095425677
1095426313 -> 1095426313
1095426743 -> 1095426743
1095425964 -> 1095425964
1095426841 -> 1095426841
1095426038 -> 1095426038
1095425875 -> 1095425781
1095425759 -> 1095425759
1095425506 -> 1095425506
1095425651 -> 1095425651
1095425522 -> 1095425522
1095425606 -> 1095425606
1095425943 -> 1095425943
1095426500 -> 1095426500
1095426847 -> 1095426847
1095425660 -> 1095425660
1095425525 -> 1095425525
1095426390 -> 1095426390
1095426308 -> 1095426308
1095426196 -> 1095426196
1095425712 -> 1095425712
1198142699 -> 1198142699
1096594587 -> 1096594587
1096590048 -> 1096590048
1096591339 -> 1096591339
1096592174 -> 1096592174
10

1096588280 -> 1096588280
1096588264 -> 1096588264
1096593791 -> 1096593791
1096588314 -> 1096588314
1096593289 -> 1096593289
1096592455 -> 1096592455
1096593354 -> 1096593354
1096588076 -> 1096588076
1096596661 -> 1096596661
1096594566 -> 1096594566
1096597062 -> 1096597062
1096589179 -> 1096589179
1096587689 -> 1096587689
1096589538 -> 1096589538
1096591070 -> 1096591070
1096589529 -> 1096589529
1096597941 -> 1096597941
1096597934 -> 1096597934
1096593407 -> 1096593407
1096596213 -> 1096596213
1096595140 -> 1096595140
1096593964 -> 1096593964
1096587226 -> 1096587226
1835348082 -> 25210887
1096593592 -> 1096593592
1096595123 -> 1096595123
1096588243 -> 1096588243
1096589560 -> 1096589560
1096595982 -> 1096595982
1096597689 -> 1096597689
1096587134 -> 1096587134
1096592595 -> 1096592595
1096589285 -> 1096589285
1096596491 -> 1096596491
1096586722 -> 1096586722
1096591801 -> 1096591801
1096597956 -> 1096597956
1096590463 -> 1096590463
1096596255 -> 1096596255
1096596739 -> 1096596739
10

StopIteration: 